

## Introduction 

The goal of this lab is to learn TCP and UDP socket programming by building a simple network application program (chat application) and analyzing TCP and UDP connections. The lab has several **milestones**. Make sure you reach each one before advancing to the next.

For delivery, see Milestone 5. 


## System Setup  

In this lab, you will build a chatroom using a client-server architecture with TCP sockets and a peer-to-peer (p2p) chat with UDP sockets, as shown in Figure 1. The overall procedure is for clients to connect to the server via TCP and register with their name. The server broadcasts information of new clients and public messages to all connected clients. Clients can also chat with each other using direct UDP connections.




|<img src="figures/setup4.jpg" width="600" height="600"/>|
|:--:| 
| **Figure 1: System Setup of chat application with socket programming** |




* __Use the same docker files as in lab2. You can run your chat server application in the "webserver" container and the chat client in "client1", "client2" containers__

* You can write the code in JupyterLab or your favorite python IDE. The code cell has the [magic command](https://ipython.readthedocs.io/en/stable/interactive/magics.html)`%%writefile`, which write the contents of the cell to a file, then copy the code files to the container either using `scp` or the mounted volumes.

* To get an overview of using sockets in python, read this [Python HOWTO](https://docs.python.org/3/howto/sockets.html) (<https://docs.python.org/3/howto/sockets.html>), which discuss TCP sockets. Additionally, you can read section 2.7 of the book.

* We will also use threading to enable running multiple sockets concurrently. You can have a look at this [tutorial](https://dzone.com/articles/python-thread-part-1) (<https://dzone.com/articles/python-thread-part-1>), to get a brief introduction to python threads.

* Finally, we will use _json_ for serializing data ([TTM4175](https://ttm4175.iik.ntnu.no/iot-http-json-2020.html) (<https://ttm4175.iik.ntnu.no/iot-http-json-2020.html>)).


# Milestone 1 -- Chat Server Application

In this milestone, you will build a chat server using the skeleton code below and the workflow, as shown in figure 2.
You are to complete the skeleton code. The places where you need to fill in code are marked with `====fill in here====`.




|<img src="figures/setup2.jpg" width="800" height="800" />|
|:--:| 
| **Figure 2: Chat application workflow of a client-server architecture using TCP sockets in python** |


In [ ]:
%%writefile chatserver.py
# Import necessary modules
import socket
from threading import Thread
import json
import sys

# Get the IP address of the server using socket 
# Don't write the IP address directly.
SERVER_IP  = #====fill in here====

# We will use a fixed port number
PORT = 5000 

# buffer size for receiving messages
RECV_BUFFER = 1024
# Dictionary for storing active Users
ACTIVE_USERS ={}  # Maps the name to the IP address

# Dictionary for storing active sockets
SOCKETS = {}  # Maps the socket to the name

# Create a server socket (TCP socket)
SERVER_SOCKET = #====fill in here====

# bind the "SERVER_SOCKET" to the SERVER_IP and PORT
# ====fill in here====

# Main thread: accepting connections from clients and making
# a child thread for every client
def accept_incoming_connections():
    while True:
        # make the SERVER_SOCKET accept  connection from a client
        client_socket, client_address = #====fill in here====
        try:
            #recieve the first message from the client_socket
            name = #====fill in here====
            name= name.decode("utf-8")
            #Storing the client name and its socket 
            SOCKETS[client_socket] = name
            ACTIVE_USERS[name] = client_address[0]
            print(f"{name} with IP address:{client_address[0]}, has connected\n")
            # Updating all clients of the current active users
            broadcast_active_users()

        except:
            # close the client_socket if an error occurs
            client_socket.close()
            print('Error connecting to a client')

        #Starting a thread for the connected client
        Thread(target=handle_client, args=(client_socket,)).start()


#Thread for handling connection to every client
def handle_client(client_socket):  
    while True:
        try:
            # receive a message from the client_socket
            msg = #====fill in here====
            msg = msg.decode("utf-8")

        except:
            # handle broken socket connection (e.g. the client pressed ctrl+c)
            client_socket.close()
            del(ACTIVE_USERS[SOCKETS[client_socket]])
            del SOCKETS[client_socket]
            broadcast_active_users()
            sys.exit()

        # len(msg)==0 when exiting wiht KeyboardInterrupt (ctrl+c)
        if len(msg)==0:
            client_socket.close()
            del(ACTIVE_USERS[SOCKETS[client_socket]])
            del(SOCKETS[client_socket])
            broadcast_active_users()
            sys.exit()
        else:
            # Attaching the name of sending client to msg
            msg = f"{SOCKETS[client_socket]}: {msg} "

            # Send the received msg to every client, except this client 
            #====fill in here====


# Function to send a message to every client, except the sending client
def broadcast_msg(sending_client,msg): 
    msg = msg.encode("utf-8")
    for client_socket in list(SOCKETS):
        # exclude sending msg to the sending_client
        if client_socket !=  sending_client: 
            try:
                # Send the msg to client_socket
                #====fill in here====

            except:
                client_socket.close()
                del(ACTIVE_USERS[SOCKETS[client_socket]])
                del (SOCKETS[client_socket])
                broadcast_active_users()


# Function to broadcast the current active users to all clients 
def broadcast_active_users():
    active_users ='!'+json.dumps(ACTIVE_USERS)
    active_users = active_users.encode("utf-8")
    for client_socket in list(SOCKETS):
        try:
            # send "active_users" to client_socket
            #====fill in here====

        except:
            client_socket.close()
            del(ACTIVE_USERS[SOCKETS[client_socket]])
            del (SOCKETS[client_socket])


if __name__ == "__main__":
    # Make the server listen for TCP connection requests
    # (5 maximum number of queued connections)
    SERVER_SOCKET.listen(5) 
    print("Waiting for connection...")
    # starting the main thread and joining all child threads
    connection_threads = Thread(target=accept_incoming_connections)
    connection_threads.start()  
    connection_threads.join()
    # after existing from the main thread, close SERVER_SOCKET
    SERVER_SOCKET.close()

In [ ]:
# TA: uncomment the following line to see the solution
# %load ./solutions/chatserver_sol.py

# Milestone 2 -- Chat Client Application

In this milestone, you will build a chat client using the skeleton code below.
You are to complete the skeleton code. You can find the python file of the skeleton code in "chatclient.py"
The places where you need to fill in code are marked with `====fill in here====`.

In the client application, a connection is first established to the chat server using a TCP socket (figure 2), and the client name is registered.
Then the application will get a dictionary of all active user names and their IP addresses.
In this way, the client can chat with another client (p2p) using UDP sockets, as shown in the workflow figure 3.




|<img src="figures/setup3.jpg" width="800" height="800" />|
|:--:| 
| **Figure 3: Chat application workflow of a peer-to-peer (p2p) architecture using UDP sockets in python** |

In [ ]:
%%writefile chatclient.py

import socket
from threading import Thread
import sys
import json

# get the IP address of the chatserver (www.ttm4200.com) 
# using your DNS server and socket methods
SERVER_IP = #====fill in here====

# Get the IP address of the client using socket methods
CLIENT_IP =  #====fill in here====

PORT = 5000 
RECV_BUFFER = 1024

# Create a TCP socket 
TCP_SOCKET = #====fill in here====

# Create a UDP socket 
UDP_SOCKET = #====fill in here====

# Make the UDP socket reusable
UDP_SOCKET.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) 

# Bind the UDP socket to the CLIENT_IP and PORT
#====fill in here====

# Dictionary for storing active users
ACTIVE_USERS = None

# Helping function to update active users
def update_active_users(msg):
    global ACTIVE_USERS
    ACTIVE_USERS = json.loads(msg[1:])


# Helping function to get the name of an active user from its IP
def get_name_from_ip_address(ip_address):
    name_list = [name for name, ip in ACTIVE_USERS.items() if ip == ip_address]
    name = name_list[0] if len(name_list) else 'Non-registered'
    return name 


# Main thread: connect to the server, then keep listening to receives msgs, 
# and send whenever there is an input
def connect_and_listen():
    try:
        # Connect TCP_SOCKET to the chat server
        #====fill in here====

        name = input("First enter your name to register it in the server: ")
        name = name.encode("utf-8")

        # send the "name" to the server
        #====fill in here====

    except:
        print("could not connect to chat server...")
        sys.exit()

    print(""" Connection Established! 
    Now you can enter '#ls' to list active users,
    '@<user1>: msg' send msg to a specific user,
    or just write your msg  to send  to all users
    *************************************************""")
    
    # Start a thread to recieve msgs on TCP socket (server connection)
    Thread(target=receive_tcp).start()
    # Start a thread to recieve msgs on udp socket (msgs from other clients, p2p)
    Thread(target=receive_udp).start()
    # Start a thread to send the message when there is input. it will send either
    # to the server (TCP socket) or a specific client using its IP (UDP socket)
    Thread(target=send).start()


def receive_tcp():
    while True:
        try:
            # Store the message received from the server
            msg = #====fill in here====
            msg = msg.decode("utf-8")

            if msg[0]== '!':
                update_active_users(msg)
            else:
                print(f"incoming msg from chatserver ({msg})")

        except OSError:  # Possibly client has left the chat.
            TCP_SOCKET.close()
            sys.exit()


def receive_udp():
    while True:
        # Receive a message from any other client (p2p chat)
        msg, address = #====fill in here====

        ip_address= address[0]
        sending_client = get_name_from_ip_address(ip_address)
        print (f"Incoming msg (P2P) from {sending_client}:{msg.decode('utf-8')}")


def send():
    while True:
        # Get input from the user
        usr_input = input(">")

        # List current active chat users
        if usr_input =='#ls':
            print(ACTIVE_USERS)

        # Send a msg to a single client (p2p) if it starts with "@"
        elif (len(usr_input) > 0) and (usr_input[0] == '@'):
            # Extract the name of the receiving_client and the msg
            temp = usr_input.split(':')
            receiving_client = temp[0][1:].strip()
            msg=':'.join(temp[1:])
            msg = msg.encode("utf-8")

            # Check if the receiving_client is in the active users
            #  (registered int the server)
            if receiving_client in ACTIVE_USERS:
                # Get the ip_address of the receiving client
                ip_address =ACTIVE_USERS[receiving_client]
                # Send "msg" to the receiving client
                #====fill in here====
            else:
                print(f"{receiving_client} is not registered")
        # Send a msg to all clients (chatroom), 
        # if it doesn't start with a specical character
        elif len(usr_input) > 0 :
            msg=usr_input
            msg = msg.encode("utf-8")
            # Send "msg" to the server (chatroom)
            #====fill in here====


if __name__ == "__main__":
    connection_threads = Thread(target=connect_and_listen)
    connection_threads.start()  # Starts the infinite loop.
    connection_threads.join()



# Milestone 3 -- Running the Chat Applications

* Copy the code file "chatserver.py" to the "webserver" container: ` scp chatserver.py ttm4200@10.20.30.3:~/`. Then access the container and run the code file: `python3 chatserver.py`.

* Configure "client1" and "client2" to use your DNS server as their nameserver (in "/etc/resolv.conf"):
   ```bash
   nameserver 10.20.30.2
   ```

* Copy then run the code file "chatclient.py" inside both "client1" and "client2" containers. Test your code correctness by creating different names for every chat client and sending messages to the chatroom and to the other client (P2P).

 ## Traffic Capture
 
* Terminate code execution inside both chat server and chat clients.

* Start a packet capture with `tcpdump` on your chat server and dumpt the capture to a "chatserver.pcap" file, then start your "chatserver.py" code.

* Start packet capturing on the two clients and dump the capture to files named "client1.pcap" and "client2.pcap", then start your "chatclient.py" code.

* Send messages to the chatroom and P2P clients and afterward stop the captures.

* __Save__ the capture files in your host machine inside the "lab3" folder.

# Milestone 4 -- Analyzing Traffic

## 4.1 TCP Connection

* Open your "chatserver.pcap" in Wireshark and display only TCP packets from "cleint1" and "client2": `(ip.addr == 10.20.30.11 || ip.addr == 10.20.30.12) && tcp`.

* Disable the display of relative sequence numbers in wireshark: **Edit**$\rightarrow$**Preferences**$\rightarrow$ exapand **Protocols** $\rightarrow$ **TCP** $\rightarrow$ untick the box for  **Relative sequence numbers**

>_By default, Wireshark keeps track of all TCP sessions and converts all Sequence Numbers, and Acknowledge Numbers into relative numbers. This means that instead of displaying the real/absolute numbers, Wireshark will display numbers relative to the first seen segment for that conversation._

* Navigate to **Statistics** $\rightarrow$ **Flow Graph**. Tich the box of **Limit to display filter**, then select **TCP Flows** from **Flow type**. This will display a sequence diagram of the TCP packets exchanged between the clients and the server. I will also display the sequence and Acknowledgement numbers on the right-hand side.

* Answer the following questions:

    * What is the sequence number of the TCP SYN segment used to initiate the first TCP connection between "client1" and the server? 
    * What is the sequence number of SYNACK segment sent by the server to the client in reply to the SYN?
    * What is the value of the ACKnowledgement field in the SYNACK segment?

In [ ]:
from test_lab3 import TestLab3
check_progress = TestLab3()
# What is the sequence number of the TCP SYN segment used to initiate the first TCP connection between "client1" and the server? 
SYN_sequence_number =  #your answer (as an interger)

# What is the sequence number of SYNACK segment sent by the server to the client in reply to the SYN?
SYNACK_sequence_number =  #your answer (as an interger)

# What is the value of the ACKnowledgement field in the SYNACK segment?
SYNACK_ACKnowledgement_number =  #your answer (as an interger)
check_progress.test_4_1(SYN_sequence_number, SYNACK_sequence_number, SYNACK_ACKnowledgement_number)

## 4.2 UDP Connection

* Open "client1.pcap" file in Wireshark and display only UDP packets from and to the other client (`ip.addr == 10.20.30.12 && udp`).Then answer the following questions:

    * Select one UDP packet from your trace (corresponding to a chat message). From this packet, determine how many fields are in the UDP header.
    * What is the length (in bytes) of a UDP header field?
    * What is the maximum number of bytes that can be included in a UDP payload?
    * What is the largest possible source port number?
    * What is the protocol number for UDP?

In [ ]:
# How many fields are in the UDP header?
number_of_fields_in_UDP_header =  #your answer (as an interger)

# What is the length (in bytes) of a UDP header field?
length_of_header_field = #your answer (as an interger)

# What is the maximum number of bytes that can be included in a UDP payload?
maximum_number_of_bytes_in_UDP_payload = #your answer (as an interger)

# What is the largest possible source port number?
largest_possible_source_port_number = #your answer (as an interger)

# What is the protocol number for UDP?
protocol_number_for_UDP = #your answer (as an interger)

check_progress.test_4_2(number_of_fields_in_UDP_header, 
                        length_of_header_field, 
                        maximum_number_of_bytes_in_UDP_payload, 
                        largest_possible_source_port_number, 
                        protocol_number_for_UDP)

# Milestone 5 -- Delivery


__Delivery Q1:__ Write an essay (in fewer than two pages, including the sequence diagram) explaining the flow of the chat application you have built. Your essay should at least address the following points:

* How hostnames and addresses get resolved.
* How endpoints for communication are created (protocol family, type of service) and associated with addresses.
* How a server socket handles multiple incoming connections. 
* Highlight the difference between connection-oriented and connectionless.
* Draw a sequence diagram of __all packet transactions__ between the clients and the server (one diagram only) when a new client connects and sends a message to the server, and a message to the other client (p2p) .

__Delivery Q2:__ We implemented the chat server without any authentication mechanism, i.e., the server socket accepts any connection. You should implement an authentication mechanism where the server only accepts authorized clients. The implementation details are up to you, but the bare minimum should include: 

    1. Clients need to register (sign in) the first time they connect, using a username and password. 
    2. Clients need to log in when they reconnect. 
    
   __Optional:__ It is recommended not to save the passwords in plaintext; instead, you should always store [hashed passwords](https://www.culttt.com/2013/01/21/why-do-you-need-to-salt-and-hash-passwords). For example, you can use [hashlib module](https://stackoverflow.com/a/56915300/893857) (<https://stackoverflow.com/a/56915300/893857>) to hash the passwords in Q2.
   
**Submit a PDF essay along with the python files**.